In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark import SparkContext, SparkConf

In [7]:
import pyspark

In [8]:
pyspark.__version__

'3.3.2'

In [ ]:
# 创建连接配置
conf = SparkConf().setAppName('sparkRddDemo').setMaster("spark://sparkstandalone:7077")
# 获取spark上下文,创建到集群的连接
sc =  SparkContext(conf=conf)

In [ ]:
# 有两种方式可以创建rdds，一种是通过上下文提供的并行化方法从一个可迭代对象或者collection中获取
# 另一种是内部的存储系统
# 下面从一个可迭代的对象中获取
data = [i for i in range(1,6)]
distData = sc.parallelize(data)
print(type(distData))
# print(distData.collect())

In [ ]:
sc.stop()

In [ ]:
sc

In [ ]:
# 从文本文件创建
distFile = sc.textFile("data.txt")
print(distFile)

In [ ]:
distFile.map(lambda s:len(s)).reduce(lambda a,b:a+b)

In [ ]:
sc.stop()

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
distData.reduce(lambda a,b:a+b).collect()